In [1]:
import pandas as pd
import numpy as np
import pickle
from face import FACE

data = pd.DataFrame(np.load('toy_example/data.npy'))
with open('toy_example/model.pkl', 'rb') as f:
    clf = pickle.load(f)

In [3]:
ce = FACE(data, clf, dist_threshold=1, density_threshold=0.001, pred_threshold=0.9)
eg = np.array([[1, 1]])
path, prob = ce.generate_counterfactual(eg)
path

,0,1
0,0.783842,0.863900
1,0.047686,1.344381
2,0.042124,2.301894
